In [10]:
import requests
import json
import pandas as pd
import itertools

In [26]:
def api_to_df(url):
    """
    accesses api as json, creates dict, and then converts dict to pandas dataframe

    args:
        - url, string url for api access
    
    returns:
        - pandas dataframe with api data
    """
    response = requests.get(url).text
    doc = json.loads(response)


    values = doc["value"]
    dim_ids = doc["id"]
    dim_sizes = doc["size"]
    dimensions = doc["dimension"]

    dim_labels = {}
    for dim in dim_ids:
        dim_info = dimensions[dim]
        label_dict = dim_info["category"]["label"]
        index_dict = dim_info["category"]["index"]
        ordered_labels = [label_dict[key] for key, _ in sorted(index_dict.items(), key=lambda x: x[1])]
        dim_labels[dim] = ordered_labels

    def flat_index_to_coords(index, sizes):
        coords = []
        for size in reversed(sizes):
            coords.insert(0, index % size)
            index = index // size
        return coords

    records = []
    for flat_key, value in values.items():
        flat_index = int(flat_key)
        coord_indices = flat_index_to_coords(flat_index, dim_sizes)

        record = {}
        for i, dim in enumerate(dim_ids):
            record[dim] = dim_labels[dim][coord_indices[i]]
        record["value"] = value
        records.append(record)

    return pd.DataFrame(records)
 
    

# Healthcare

In [35]:
healthcare = api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/edat_lfse_03?format=JSON&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=PC&sex=M&sex=F&age=Y20-24&age=Y25-34&age=Y35-44&age=Y45-54&age=Y55-64&isced11=ED0-2&isced11=ED3_4&isced11=ED5-8&lang=en')
healthcare

,freq,sex,age,unit,isced11,geo,time,value
0,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Austria,2022,12.0
1,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Belgium,2022,9.8
2,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Bulgaria,2022,13.0
3,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Cyprus,2022,8.9
4,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Czechia,2022,8.5
...,...,...,...,...,...,...,...,...
805,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Portugal,2022,13.5
806,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Romania,2022,10.1
807,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Sweden,2022,28.5
808,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Slovenia,2022,24.4


In [30]:
def clean_healthcare_df(df):
    df = df.drop(columns=["freq", "unit", "time"])
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    df_encoded = pd.get_dummies(df, columns=["sex", "age", "isced11"])
    pivot_df = df_encoded.pivot_table(index="geo", aggfunc="mean")
    pivot_df = pivot_df.reset_index()
    return pivot_df

In [33]:
clean_healthcare_df(healthcare)

,geo,age_From 20 to 24 years,age_From 25 to 34 years,age_From 35 to 44 years,age_From 45 to 54 years,age_From 55 to 64 years,"isced11_Less than primary, primary and lower secondary education (levels 0-2)",isced11_Tertiary education (levels 5-8),isced11_Upper secondary and post-secondary non-tertiary education (levels 3 and 4),sex_Females,sex_Males,value
0,Austria,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.330000
1,Belgium,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.323333
2,Bulgaria,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.326667
3,Croatia,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.333333
4,Cyprus,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.343333
5,Czechia,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.326667
6,Denmark,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.330000
7,Estonia,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.340000
8,Finland,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.333333
9,France,0.2,0.2,0.2,0.2,0.2,0.333333,0.333333,0.333333,0.5,0.5,33.340000


# Education

In [19]:
api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/edat_lfse_03?format=JSON&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=PC&sex=M&sex=F&age=Y20-24&age=Y25-34&age=Y35-44&age=Y45-54&age=Y55-64&isced11=ED0-2&isced11=ED3_4&isced11=ED5-8&lang=en')


,freq,sex,age,unit,isced11,geo,time,value
0,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Austria,2022,12.0
1,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Belgium,2022,9.8
2,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Bulgaria,2022,13.0
3,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Cyprus,2022,8.9
4,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Czechia,2022,8.5
...,...,...,...,...,...,...,...,...
805,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Portugal,2022,13.5
806,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Romania,2022,10.1
807,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Sweden,2022,28.5
808,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Slovenia,2022,24.4


# Safety

In [20]:
api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/crim_off_cat?format=JSON&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=NR&unit=P_HTHAB&iccs=ICCS0101&iccs=ICCS0102&iccs=ICCS020111&iccs=ICCS020221&iccs=ICCS0301&iccs=ICCS03011&iccs=ICCS03012&iccs=ICCS0302&iccs=ICCS030221&iccs=ICCS0401&iccs=ICCS0501&iccs=ICCS05012&iccs=ICCS0502&iccs=ICCS05021&iccs=ICCS0601&iccs=ICCS0701&iccs=ICCS0703&iccs=ICCS07031&iccs=ICCS07041&iccs=ICCS0903&iccs=ICCS09051&lang=en')


,freq,iccs,unit,geo,time,value
0,Annual,Intentional homicide,Number,Austria,2022,65.00
1,Annual,Intentional homicide,Number,Belgium,2022,185.00
2,Annual,Intentional homicide,Number,Bulgaria,2022,76.00
3,Annual,Intentional homicide,Number,Cyprus,2022,7.00
4,Annual,Intentional homicide,Number,Czechia,2022,79.00
...,...,...,...,...,...,...
1045,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Poland,2022,2.88
1046,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Portugal,2022,0.06
1047,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Romania,2022,2.71
1048,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Slovenia,2022,0.00


# Culture

In [21]:
api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/ilc_scp03?format=JSON&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=PC&unit=PC&frequenc=NM12&frequenc=1-3&frequenc=GE4&frequenc=NM12&frequenc=1-3&frequenc=GE4&acl00=AC52A&acl00=AC522A&acl00=AC523H&acl00=AC525&acl00=AC52A&acl00=AC522A&acl00=AC523H&acl00=AC525&isced11=ED0-2&isced11=ED3_4&isced11=ED5-8&isced11=ED0-2&isced11=ED3_4&isced11=ED5-8&age=Y16-24&age=Y25-34&age=Y35-44&age=Y45-64&age=Y_GE65&age=Y16-24&age=Y25-34&age=Y35-44&age=Y45-64&age=Y_GE65&sex=M&sex=F&sex=M&sex=F&lang=en')


,freq,frequenc,acl00,isced11,age,sex,unit,geo,time,value
0,Annual,From 1 to 3 times,"Live performances (theatre, concerts, ballet)","Less than primary, primary and lower secondary...",From 16 to 24 years,Females,Percentage,Austria,2022,25.4
1,Annual,From 1 to 3 times,"Live performances (theatre, concerts, ballet)","Less than primary, primary and lower secondary...",From 16 to 24 years,Females,Percentage,Belgium,2022,34.0
2,Annual,From 1 to 3 times,"Live performances (theatre, concerts, ballet)","Less than primary, primary and lower secondary...",From 16 to 24 years,Females,Percentage,Bulgaria,2022,9.9
3,Annual,From 1 to 3 times,"Live performances (theatre, concerts, ballet)","Less than primary, primary and lower secondary...",From 16 to 24 years,Females,Percentage,Cyprus,2022,39.0
4,Annual,From 1 to 3 times,"Live performances (theatre, concerts, ballet)","Less than primary, primary and lower secondary...",From 16 to 24 years,Females,Percentage,Czechia,2022,44.2
...,...,...,...,...,...,...,...,...,...,...
9223,Annual,Not in the last 12 months,"Cultural activities (cinema, live performances...",Tertiary education (levels 5-8),65 years or over,Males,Percentage,Portugal,2022,34.0
9224,Annual,Not in the last 12 months,"Cultural activities (cinema, live performances...",Tertiary education (levels 5-8),65 years or over,Males,Percentage,Romania,2022,71.8
9225,Annual,Not in the last 12 months,"Cultural activities (cinema, live performances...",Tertiary education (levels 5-8),65 years or over,Males,Percentage,Sweden,2022,33.3
9226,Annual,Not in the last 12 months,"Cultural activities (cinema, live performances...",Tertiary education (levels 5-8),65 years or over,Males,Percentage,Slovenia,2022,49.0


# Environmental Consciousness

In [22]:
api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/sdg_13_10?format=JSON&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=T_HAB&unit=I90&src_crf=TOTXMEMO&src_crf=TOTX4_MEMO&lang=en')

,freq,src_crf,unit,geo,time,value
0,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Austria,2022,92.3
1,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Belgium,2022,70.7
2,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Bulgaria,2022,59.0
3,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Cyprus,2022,149.5
4,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Czechia,2022,58.6
...,...,...,...,...,...,...
103,Annual,Total (excluding memo items),Tonnes per capita,Portugal,2022,5.3
104,Annual,Total (excluding memo items),Tonnes per capita,Romania,2022,3.4
105,Annual,Total (excluding memo items),Tonnes per capita,Sweden,2022,1.0
106,Annual,Total (excluding memo items),Tonnes per capita,Slovenia,2022,5.4


# Public Infrastructure

In [23]:

api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/rail_pa_total?format=JSON&time=2022&geo=EU27_2020&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=LV&geo=LT&geo=LU&geo=HU&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=MIO_PKM&unit=THS_PAS&lang=en')

,freq,unit,geo,time,value
0,Annual,Millions of passenger-kilometres,Austria,2022,12855
1,Annual,Millions of passenger-kilometres,Bulgaria,2022,1600
2,Annual,Millions of passenger-kilometres,Czechia,2022,9394
3,Annual,Millions of passenger-kilometres,Germany,2022,92313
4,Annual,Millions of passenger-kilometres,Denmark,2022,5822
5,Annual,Millions of passenger-kilometres,Estonia,2022,382
6,Annual,Millions of passenger-kilometres,Greece,2022,1117
7,Annual,Millions of passenger-kilometres,Spain,2022,27489
8,Annual,Millions of passenger-kilometres,European Union - 27 countries (from 2020),2022,386741
9,Annual,Millions of passenger-kilometres,Finland,2022,4572


# Quality of Life Index

In [24]:
api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/ilc_pw01?format=JSON&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=RTG&statinfo=AVG&isced11=ED0-2&isced11=ED3_4&isced11=ED5-8&life_sat=LIFE&sex=M&sex=F&age=Y16-24&age=Y25-34&age=Y35-49&age=Y50-64&age=Y_GE65&lang=en')

,freq,statinfo,unit,isced11,life_sat,sex,age,geo,time,value
0,Annual,Average,Rating (0-10),"Less than primary, primary and lower secondary...",Life as a whole,Females,From 16 to 24 years,Austria,2022,8.1
1,Annual,Average,Rating (0-10),"Less than primary, primary and lower secondary...",Life as a whole,Females,From 16 to 24 years,Belgium,2022,7.7
2,Annual,Average,Rating (0-10),"Less than primary, primary and lower secondary...",Life as a whole,Females,From 16 to 24 years,Bulgaria,2022,5.8
3,Annual,Average,Rating (0-10),"Less than primary, primary and lower secondary...",Life as a whole,Females,From 16 to 24 years,Cyprus,2022,7.7
4,Annual,Average,Rating (0-10),"Less than primary, primary and lower secondary...",Life as a whole,Females,From 16 to 24 years,Czechia,2022,8.0
...,...,...,...,...,...,...,...,...,...,...
788,Annual,Average,Rating (0-10),Tertiary education (levels 5-8),Life as a whole,Males,65 years or over,Portugal,2022,7.4
789,Annual,Average,Rating (0-10),Tertiary education (levels 5-8),Life as a whole,Males,65 years or over,Romania,2022,8.0
790,Annual,Average,Rating (0-10),Tertiary education (levels 5-8),Life as a whole,Males,65 years or over,Sweden,2022,7.8
791,Annual,Average,Rating (0-10),Tertiary education (levels 5-8),Life as a whole,Males,65 years or over,Slovenia,2022,7.9
